Choosing the right hyperparameters is sort of annoying but necessary process for any data science projects. Some model may be more sensitive to some hyperparameters. 

In [ ]:
def run_experiemnts(config,)

In [2]:
from hedgeable_ai.optimizer.tuner.space import DesignSpace

/home/tom/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:

mixed_domain =[{'name': 'var1', 'type': 'continuous', 'domain': (-5,5),'dimensionality': 3},
               {'name': 'var3', 'type': 'discrete', 'domain': (3,8,10),'dimensionality': 2},
               {'name': 'var4', 'type': 'categorical', 'domain': ('hey', 'what up', 'oops'),'dimensionality': 1},
               {'name': 'var5', 'type': 'integer', 'domain': (1, 10), 'scale':'log'}]

space = DesignSpace(mixed_domain)

{'name': 'var1', 'type': 'continuous', 'domain': (-5, 5), 'dimensionality': 3}
3
{'name': 'var3', 'type': 'discrete', 'domain': (3, 8, 10), 'dimensionality': 2}
2
{'name': 'var4', 'type': 'categorical', 'domain': ('hey', 'what up', 'oops'), 'dimensionality': 1}
1
{'name': 'var5', 'type': 'integer', 'domain': (1, 10), 'scale': 'log'}
1


In [15]:
space.get_bounds()

[(-5, 5),
 (-5, 5),
 (-5, 5),
 (3, 10),
 (3, 10),
 (0, 1),
 (0, 1),
 (0, 1),
 (0.0, 1.0)]

In [14]:
len(space.config_space_expanded)

7

In [4]:
x = [0, 0, 0, 8, 1, 'hey', 10]

model_x = space.objective_to_model([x])
print(model_x)
obj_x = space.model_to_objective([model_x])
print(obj_x)

[0. 0. 0. 8. 1. 1. 0. 0. 1.]
[0.0, 0.0, 0.0, 8, 3, 'hey', 2]


In [5]:
x = space.space_expanded[-1]

In [6]:
x.scale

In [14]:
x.get('scale', None)

'log'

In [6]:
import math
math.log10(2)

0.3010299956639812

In [5]:
1e-5

1e-05

In [1]:
from hedgeable_ai.optimizer.tuner import RandomOptimizer


mixed_domain =[{'name': 'var1', 'type': 'continuous', 'domain': (-5,5),'dimensionality': 3},
               {'name': 'var3', 'type': 'discrete', 'domain': (3,8,10),'dimensionality': 2},
               {'name': 'var4', 'type': 'categorical', 'domain': ('hey', 'what up', 'oops'),'dimensionality': 1},
               {'name': 'var5', 'type': 'integer', 'domain': (1, 10), 'scale':'log'}]

opt = RandomOptimizer(score_func=lambda x: x, params_conf=mixed_domain)

/home/tom/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


{'name': 'var1', 'type': 'continuous', 'domain': (-5, 5), 'dimensionality': 3}
3
{'name': 'var3', 'type': 'discrete', 'domain': (3, 8, 10), 'dimensionality': 2}
2
{'name': 'var4', 'type': 'categorical', 'domain': ('hey', 'what up', 'oops'), 'dimensionality': 1}
1
{'name': 'var5', 'type': 'integer', 'domain': (1, 10), 'scale': 'log'}
1


In [2]:
x = [0., 0., 0., 8., 1., 1., 0., 0., 1.]
param = opt.vec2params(x)

[0.0, 0.0, 0.0, 8, 3, 'hey', 10]


In [3]:
param

{'var1_1': 0.0,
 'var1_2': 0.0,
 'var1_3': 0.0,
 'var3_1': 8,
 'var3_2': 3,
 'var4_1': 'hey',
 'var5': 10}

In [4]:
opt.params2vec(param)

array([0., 0., 0., 8., 3., 1., 0., 0., 1.])

In [5]:
opt.design_space.dimensionality

7

In [6]:
opt.design_space.model_dimensionality

9